In [ ]:
import numpy as np

from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient("ncsa_teststand_efd")

In [ ]:
si_df = await client.select_top_n("lsst.sal.CCCamera.logevent_startIntegration",
                                  ["private_sndStamp", "additionalValues"], 15)

In [ ]:
eoit_df = await client.select_top_n("lsst.sal.CCCamera.logevent_endOfImageTelemetry",
                                    ["private_sndStamp", "additionalValues"], 15)

In [ ]:
image_type = "FLAT"
si_image_type = si_df["additionalValues"].str.contains(image_type)
eoit_image_type = eoit_df["additionalValues"].str.contains(image_type)

In [ ]:
si_timestamps = np.flip(si_df["private_sndStamp"][si_image_type].values)
eoit_timestamps = np.flip(eoit_df["private_sndStamp"][eoit_image_type].values)

In [ ]:
eoit_timestamps[:-1] - si_timestamps[1:]

In [ ]:
eoit_timestamps - si_timestamps